# **Data Wrangling**
En este notebook realizo las tareas identificadas en el **EDA**, entre ellas:
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Formateo a `snake_case` y eliminación de tildes
- Pasado de USD a pesos

## 1. Lectura de archivos _staging_ e importación de librerias
- Definición de la carpeta raíz e importación de librerias

In [58]:

import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)
    
# Importado de módulos y librerías
from scripts.utils import pasar_snake_case, aplicar_funcion_df, transformar_unidades, traductor_cols
import pandas as pd
import json

- Lectura de archivos

In [59]:
dir_staging = r'..\data\staging'

# Lectura de CSVs 
df_orders = pd.read_csv(dir_staging + r'\orders.csv')
df_items = pd.read_csv(dir_staging + r'\items_traducido.csv')
df_indec_temporal = pd.read_csv(dir_staging + r'\indec.csv')

- Asignación de columnas

In [60]:
df_indec = pd.DataFrame()

# Asignación de columnas
df_indec['variedad'] = df_indec_temporal.iloc[:, 0]
df_indec['unidad_de_medida'] = df_indec_temporal.iloc[:, 1]
df_indec['precio'] = df_indec_temporal.iloc[:, 2]

# Revisión rápida
print(df_indec.isnull().sum().sum()) 

10


## 2. Limpieza de datos

- Eliminación de valores nulos

In [61]:
df_items = df_items.dropna() # aunque ya son eliminados en "traducir_df.py"
df_indec = df_indec.dropna() 

## 3. Conversiones de datos

- Traducción de columnas del inglés al español

In [62]:
# Traduzco las columnas
for df in [df_orders, df_items]:
    df.columns = [traductor_cols(col) for col in df.columns.tolist()]

> **Nota:** La traducción del DataFrame general es ejecutada en un script aparte (`traducir_df.py`). Para más información, acceder a él.

- Pasado a `snake_case` con una función auxiliar

In [63]:
# Convertir valores a snake_case solo para ciertas columnas
aplicar_funcion_df(df_items, ["categoria", "subcategoria", "nombre_producto"], pasar_snake_case)
aplicar_funcion_df(df_indec, ["variedad", "unidad_de_medida", "precio"], pasar_snake_case) # la columna precio es una string, reemplazo "," y "."

- Conversión de precios y unidades de medida a kilogramos

In [64]:
# Conversión a float
df_indec["precio"] = df_indec["precio"].astype(float)

# Utilizo función auxiliar que se encarga de pasar precios de diversas unidades/medidas a kg
df_indec = transformar_unidades(df_indec)

c:\Users\User\Desktop\analisis-de-desperdicios\scripts\utils.py:103: UserWarning: Valores no reconocidos en transformar_unidades: [('hamburguesas_congeladas', 'env_4_u', 'Unidad desconocida'), ('salchicha_tipo_viena', 'env_6_u', 'Unidad desconocida'), ('huevos_de_gallina', 'docena', 'Formato desconocido'), ('polvo_para_flan', '8_porciones', 'Unidad desconocida'), ('panales_descartables', '10_u', 'Unidad desconocida')]
  warnings.warn(f"Valores no reconocidos en transformar_unidades: {avisos}")


- Conversión de tipo de cambio (dólares a pesos)

In [65]:

# Obtengo valor del dólar
with open(dir_staging + r"\dolar_blue.txt", "r") as db:
    valor_dolar = float(db.readline())

# Aplico cálculos
aplicar_funcion_df(df_items, ["precio", "costo_produccion"], lambda cant: round((cant * valor_dolar), 1))


## 5. Cálculo de ganancias 
Tanto en: 
- pesos $ 
- porcentajes %, donde el porcentaje sería en relación con la ganancia total -> ganancia total es el 100%, entonces ganancia neta es X%

In [ ]:
# Uno las tablas
df_ventas = df_orders.merge(df_items, on="item", how="inner")

ganancia_total = round(((df_ventas["cantidad"] * df_ventas["precio"]).sum()), 1)
gasto_total = round(((df_ventas["cantidad"] * df_ventas["costo_produccion"]).sum()), 1)

# Cálculo monetario $
ganancia_neta = round((ganancia_total - gasto_total), 1)

# Cálculo percentil %
porcentaje_neto = round(((ganancia_neta * 100) / ganancia_total), 2)

print(f'Las ganancias desde el {df_ventas["fecha_orden"].min()} hasta el {df_ventas["fecha_orden"].max()}:')
print(f' - ${ganancia_neta} pesos')
print(f' - Con un promedio de {porcentaje_neto}% de ganancia por cada venta')

Las ganancias desde el 2022-01-01 hasta el 2022-12-31:
 - $854914832.2 pesos
 - Con un promedio de 15.73% de ganancia por cada venta


## 6. Cálculo de pérdidas
- Utilizando un promedio de pérdidas por fugas de un ~10% (estimado en `data\README.md`) donde está aplicado (el 10%) al costo de los insumos
- Y estimando que con una buena gestión podría reducirse en un 5%

In [ ]:
perdida_optimizada = gasto_total * 0.05 
gasto_optimizado = round((gasto_total - (gasto_total * 0.05)), 1)

# Cálculo monetario $
ganancia_neta_opt = round((ganancia_total - gasto_optimizado), 1)

# Cálculo percentil %
porcentaje_neto_opt = round(((ganancia_neta_opt * 100) / ganancia_total), 2)

# Ganancias posibles
diferencia_ganancia = round((ganancia_neta_opt - ganancia_neta), 1)
diferencia_porcentaje = round((porcentaje_neto_opt - porcentaje_neto), 2)

print(f'Las ganancias optmizadas desde el {df_ventas["fecha_orden"].min()} hasta el {df_ventas["fecha_orden"].max()}:')
print(f' - ${ganancia_neta_opt} pesos -> Diferencia de {diferencia_ganancia}')
print(f' - Con un promedio de {porcentaje_neto_opt}% de ganancia por cada venta -> Diferencia del {diferencia_porcentaje}%')

Las ganancias optmizadas desde el 2022-01-01 hasta el 2022-12-31:
 - $1083938088.1 pesos -> Diferencia de 229023255.9
 - Con un promedio de 19.94% de ganancia por cada venta -> Diferencia del 4.21%


## 5. Almacenado
- DataFrames en CSV
- KPIs y datos clave en JSON

In [68]:
clean_dir = r"..\data\clean"

# Almacenado de los DataFrames en formato CSV
df_orders.to_csv(clean_dir + r"\orders.csv", index=False)
df_items.to_csv(clean_dir + r"\items.csv", index=False)
df_indec.to_csv(clean_dir + r"\indec.csv", index=False)

kpis = {
    "ganancia_total": ganancia_total,
    "gasto_total": gasto_total,
    "ganancia_neta": ganancia_neta,
    "gasto_optimizado": gasto_optimizado,
    "ganancia_neta_opt": ganancia_neta_opt,
    "porcentaje_neto_opt": porcentaje_neto_opt,
    "diferencia_ganancia": diferencia_ganancia,
    "diferencia_porcentaje": diferencia_porcentaje
}

# Almacenado en JSON
with open(clean_dir + r"\kpis.json", "w") as f:
    json.dump(kpis, f)